In [1]:
'''
Notebook for FIG3 - Consider the case of the Punisher circuit with two toggle switches
'''
# By Kirill Sechkar

# PACKAGE IMPORTS 
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
from math import pi
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS
import synthetic_circuits as circuits
from cell_model import *

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het,\
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.twotoggles_punisher_initialise,
    circuits.twotoggles_punisher_ode,
    circuits.twotoggles_punisher_F_calc,
    circuits.twotoggles_punisher_eff_m_het_div_k_het,
    par, init_conds)  # load the circuit

In [3]:
# PARAMETERISE THE CIRCUIT (DEFAULT)

# TOGGLE SWITCHES
for togswitchnum in (1, 2):  # cycle through toggle switches
    for toggenenum in (1, 2):  # cycle through the genes of the current switch
        par['c_tog' + str(togswitchnum) + str(toggenenum)] = 1  # copy no. (nM)
        par['a_tog' + str(togswitchnum) + str(toggenenum)] = 1e5/2  # promoter strength (unitless)

        # transcription regulation function
        reg_func_string = 'dna(tog' + str(togswitchnum) + str(toggenenum) + '):p_tog' + str(togswitchnum) + str(
            (toggenenum - 2) % 2 + 1)  # dna(rep1):p_rep3, dna(rep2):p_rep1 or dna(rep3):p_rep2
        par['K_' + reg_func_string] = 2500  # half-saturation constant
        par['eta_' + reg_func_string] = 2  # Hill coefficient
        par['baseline_tog' + str(togswitchnum) + str(
            toggenenum)] = 0.025  # baseline transcription activation due to leakiness
        par['p_tog' + str(togswitchnum) + str(
            toggenenum) + '_ac_frac'] = 1  # active fraction of protein (i.e. share of molecules not bound by the inducer)
    # break symmetry for each of the toggle switches
    init_conds['m_tog' + str(togswitchnum) + '1'] = 4000

# PUNISHER
# switch gene conc
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch']=0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0#0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 500.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 10.0  # gene concentration (nM)
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['baseline_switch_alt'] = 0
par['p_switch_ac_frac'] = 0.87  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

In [4]:
# SET UP THE DETERMINISTIC SIMULATION PARAMETERS

# diffrax simulator
savetimestep = 0.05  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver

# SIMULATION TIME FRAMES
# getting the initial state
tf_prepun = (0,25+savetimestep/2) # time frame for simulation before punisher comes online - to avoid premature tiggering
tf_pun = (25,50+savetimestep/2) # time frame for simulation after punisher comes online
# scenario 1: mutating a toggle
tf_mut=(50,85+savetimestep/2)
# scenario 2: flipping both toggles with an inducer pulse
tf_pulse = (50,50.75+savetimestep/2) # time frame for inducer pulse that flips the toggle
tf_afterpulse = (50.75,85+savetimestep/2) # time frame for simulation after the pulse

# flipping a toggle: inducer pulse parameters
pulse_ptog11_acfrac = 0.0 # active fraction of p_tog11 during the inducer pulse
pulse_ptog21_acfrac = 0.0 # active fraction of p_tog21 during the inducer pulse

In [6]:
# SIMULATE THE MUTATION OF ONE TOGGLE SWITCH

par_onemut = par.copy()

 # initial simulation to get the steady state without gene expression loss
ac_frac = par_onemut['p_switch_ac_frac']
par_onemut['p_switch_ac_frac'] = 0.0  # first with punisher inactive to avoid it being triggered
sol = ode_sim(par_onemut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_onemut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf_prepun, jnp.arange(tf_prepun[0], tf_prepun[1], savetimestep),  # time frame and time axis for saving the system's state
              rtol, atol)  # relative and
ts = np.array(sol.ts)
xs = np.array(sol.ys)
par_onemut['p_switch_ac_frac'] = ac_frac
sol_punisher_active = ode_sim(par_onemut,  # dictionary with model parameters
                              ode_with_circuit,  # ODE function for the cell with synthetic circuit
                              sol.ys[-1, :],
                              # initial condition VECTOR
                              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                              cellmodel_auxil.synth_gene_params_for_jax(par_onemut, circuit_genes),
                              # synthetic gene parameters for calculating k values
                              tf_pun, jnp.arange(tf_pun[0], tf_pun[1]+savetimestep/2, savetimestep),
                              # time frame and time axis for saving the system's state
                              rtol, atol)  # relative and
ts = np.concatenate((ts, np.array(sol_punisher_active.ts)), axis=0)
xs = np.concatenate((xs, np.array(sol_punisher_active.ys)), axis=0)

par_onemut['func_tog11'] = 0.0 
par_onemut['func_tog12'] = 0.0
x0_mut = sol_punisher_active.ys[-1, :]  # simulation will resume from the last time point
sol_mut = ode_sim(par_onemut,  # dictionary with model parameters
                        ode_with_circuit,  # ODE function for the cell with synthetic circuit
                        x0_mut,  # initial condition VECTOR
                        len(circuit_genes), len(circuit_miscs), circuit_name2pos,  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                        cellmodel_auxil.synth_gene_params_for_jax(par_onemut, circuit_genes), # synthetic gene parameters for calculating k values
                        tf_mut, jnp.arange(tf_mut[0], tf_mut[1], savetimestep),  # time frame and time axis for saving the system's state
                        rtol, atol)  # relative and absolute tolerances
onemut_ts = np.concatenate((ts, np.array(sol_mut.ts)), axis=0)
onemut_xs = np.concatenate((xs, np.array(sol_mut.ys)), axis=0)

# get growth rates
_, onemut_ls, _, _, _, _, _, _ =cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(onemut_ts, onemut_xs, par_onemut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                             circuit_eff_m_het_div_k_het)

In [7]:
# ONE TOGGLE MUTATED - PLOTS

# TOGGLE PROTEIN LEVELS
onemut_togprot_figure = bkplot.figure(
        frame_width=200,
        frame_height=125,
        x_axis_label="Time since mutation, h",
        y_axis_label="Protein conc., nM",
        x_range=(-5,tf_mut[1]-tf_mut[0]),
        y_range=(0,5.2e4),
        tools="box_zoom,pan,hover,reset,save"
    )

# add shading to show when synthetic gene expression loss occurs
onemut_togprot_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_mut[1] - tf_mut[0], tf_mut[1] - tf_mut[0]],
                            ys=[0, 1.1e5, 1.1e5, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
onemut_togprot_figure.add_layout(bkmodels.Label(x=0, y=5.2e4,
                                x_offset=2, y_offset=-12,
                                text='Toggle 1 genes mutated',
                                text_font_size='8pt',
                                text_align='left'))

# plot the toggle switch protein concentrations for tog11 and tog21
onemut_togprot_figure.line(onemut_ts - tf_pulse[0], onemut_xs[:, circuit_name2pos['p_tog11']], line_width=2, line_color=bkRGB(100, 149, 237),
           legend_label='tog11')
onemut_togprot_figure.line(onemut_ts - tf_pulse[0], onemut_xs[:, circuit_name2pos['p_tog21']], line_width=2, line_color=bkRGB(207, 181, 59),
           legend_label='tog21')

# plot the toggle switch protein concentrations for tog12 and tog22
onemut_togprot_figure.line(onemut_ts - tf_pulse[0], onemut_xs[:, circuit_name2pos['p_tog12']], line_width=2, line_color=bkRGB(127, 255, 212),
           legend_label='tog12')
onemut_togprot_figure.line(onemut_ts - tf_pulse[0], onemut_xs[:, circuit_name2pos['p_tog22']], line_width=2, line_color=bkRGB(252, 194, 0),
           legend_label='tog22')

# legend formatting
onemut_togprot_figure.legend.location = "bottom_right"
onemut_togprot_figure.legend.label_text_font_size = "8pt"
onemut_togprot_figure.legend.padding = 3
onemut_togprot_figure.legend.spacing = 2
onemut_togprot_figure.legend.glyph_width = 10
onemut_togprot_figure.legend.margin = 7


# set fonts
onemut_togprot_figure.xaxis.axis_label_text_font_size = "8pt"
onemut_togprot_figure.xaxis.major_label_text_font_size = "8pt"
onemut_togprot_figure.yaxis.axis_label_text_font_size = "8pt"
onemut_togprot_figure.yaxis.major_label_text_font_size = "8pt"
onemut_togprot_figure.yaxis[0].formatter = bkmodels.PrintfTickFormatter(format="%.1e")

# GROWTH RATES AND INTEGRASE LEVELS
onemut_grint_figure = bkplot.figure(
        frame_width=200,
        frame_height=125,
        x_axis_label="Time since mutation, h",
        y_axis_label="Cell growth rate, 1/h",
        x_range=(-5, tf_afterpulse[1] - tf_pulse[0]),
        y_range=(0.25, 1.75),
        tools="box_zoom,pan,hover,reset,save"
    )

onemut_grint_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_mut[1] - tf_mut[0], tf_mut[1] - tf_mut[0]],
                            ys=[0, 1.75, 1.75, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
onemut_grint_figure.add_layout(bkmodels.Label(x=0, y=1.75,
                                x_offset=2, y_offset=-12,
                                text='Toggle 1 genes mutated',
                                text_font_size='8pt',
                                text_align='left'))
# plot the growth rate
onemut_grint_figure.line(onemut_ts - tf_mut[0], onemut_ls, line_width=2, line_color=bkRGB(0, 0, 0), legend_label='Growth rate')

# create an extra  y range for plotting integrase protein concentrations
onemut_grint_figure.extra_y_ranges = {"p_int": bkmodels.Range1d(start=0, end=125)}
onemut_grint_figure.add_layout(bkmodels.LinearAxis(y_range_name="p_int",
                                     axis_label="Integrase conc., nM",
                                     axis_line_color=bkRGB(255, 103, 0),
                                     major_tick_line_color=bkRGB(255, 103, 0),
                                     minor_tick_line_color=bkRGB(255, 103, 0)),
                 'right')  # add the alternative axis label to the figure

# plot the integrase protein concentrations
onemut_grint_figure.line(onemut_ts - tf_mut[0], onemut_xs[:, circuit_name2pos['p_int']], line_width=2, line_color=bkRGB(255, 103, 0),
           y_range_name="p_int", legend_label="Int. conc.")

# set fonts
onemut_grint_figure.xaxis.axis_label_text_font_size = "8pt"
onemut_grint_figure.xaxis.major_label_text_font_size = "8pt"
onemut_grint_figure.yaxis.axis_label_text_font_size = "8pt"
onemut_grint_figure.yaxis.major_label_text_font_size = "8pt"

# legend formatting
onemut_grint_figure.legend.location = "top_right"
onemut_grint_figure.legend.label_text_font_size = "8pt"
onemut_grint_figure.legend.padding = 3
onemut_grint_figure.legend.spacing = 2
onemut_grint_figure.legend.glyph_width = 10
onemut_grint_figure.legend.margin = 12

# show plots
onemut_togprot_figure.output_backend = "svg"
onemut_grint_figure.output_backend = "svg"
bkplot.show(bklayouts.column([onemut_togprot_figure, onemut_grint_figure]))

# save plots
# onemut_togprot_figure.output_backend = "svg"
# onemut_grint_figure.output_backend = "svg"
# bkplot.output_file("twotoggle_onemut_plots.html")
# bkplot.save(bklayouts.column([onemut_togprot_figure, onemut_grint_figure]))

In [9]:
# SIMULATE THE FLIPPING OF BOTH TOGGLES

par_flip = par.copy()

 # initial simulation to get the steady state without gene expression loss
ac_frac = par_flip['p_switch_ac_frac']
par_flip['p_switch_ac_frac'] = 0.0  # first with punisher inactive to avoid it being triggered
sol = ode_sim(par_flip,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_flip, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf_prepun, jnp.arange(tf_prepun[0], tf_prepun[1]+savetimestep/2, savetimestep),  # time frame and time axis for saving the system's state
              rtol, atol)  # relative and
ts = np.array(sol.ts)
xs = np.array(sol.ys)
par_flip['p_switch_ac_frac'] = ac_frac
sol_punisher_active = ode_sim(par_flip,  # dictionary with model parameters
                              ode_with_circuit,  # ODE function for the cell with synthetic circuit
                              sol.ys[-1, :],
                              # initial condition VECTOR
                              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                              cellmodel_auxil.synth_gene_params_for_jax(par_flip, circuit_genes),
                              # synthetic gene parameters for calculating k values
                              tf_pun, jnp.arange(tf_pun[0], tf_pun[1]+savetimestep/2, savetimestep),
                              # time frame and time axis for saving the system's state
                              rtol, atol)  # relative and
ts = np.concatenate((ts, np.array(sol_punisher_active.ts)), axis=0)
xs = np.concatenate((xs, np.array(sol_punisher_active.ys)), axis=0)

# simulate toggle switch flipping by inducer pulse
orig_ptog11_acfrac = par_flip['p_tog11_ac_frac']  # save the original active fraction of p_tog11
orig_ptog21_acfrac = par_flip['p_tog21_ac_frac']  # save the original active fraction of p_tog21
par_flip['p_tog11_ac_frac'] = pulse_ptog11_acfrac  # set the active fraction of p_tog11 to the pulse value
par_flip['p_tog21_ac_frac'] = pulse_ptog21_acfrac  # set the active fraction of p_tog21 to the pulse value
x0_pulse = sol_punisher_active.ys[-1, :]  # simulation will resume from the last time point
sol_pulse = ode_sim(par_flip,  # dictionary with model parameters
                        ode_with_circuit,  # ODE function for the cell with synthetic circuit
                        x0_pulse,  # initial condition VECTOR
                        len(circuit_genes), len(circuit_miscs), circuit_name2pos,  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                        cellmodel_auxil.synth_gene_params_for_jax(par_flip, circuit_genes), # synthetic gene parameters for calculating k values
                        tf_pulse, jnp.arange(tf_pulse[0], tf_pulse[1]+savetimestep/2, savetimestep),  # time frame and time axis for saving the system's state
                        rtol, atol)  # relative and absolute tolerances
flip_ts = np.concatenate((ts, np.array(sol_pulse.ts)), axis=0)
flip_xs = np.concatenate((xs, np.array(sol_pulse.ys)), axis=0)

# after the pulse, inducer levels return to the original state
par_flip['p_tog11_ac_frac'] = orig_ptog11_acfrac  # set the active fraction of p_tog11 to the pulse value
par_flip['p_tog21_ac_frac'] = orig_ptog21_acfrac  # set the active fraction of p_tog21 to the pulse value
x0_afterpulse = sol_pulse.ys[-1, :]  # simulation will resume from the last time point
sol_afterpulse = ode_sim(par_flip,  # dictionary with model parameters
                        ode_with_circuit,  # ODE function for the cell with synthetic circuit
                        x0_afterpulse,  # initial condition VECTOR
                        len(circuit_genes), len(circuit_miscs), circuit_name2pos,  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                        cellmodel_auxil.synth_gene_params_for_jax(par_flip, circuit_genes),  # synthetic gene parameters for calculating k values
                        tf_afterpulse, jnp.arange(tf_afterpulse[0], tf_afterpulse[1], savetimestep),  # time frame and time axis for saving the system's state
                        rtol, atol)  # relative and absolute tolerances
flip_ts = np.concatenate((flip_ts, np.array(sol_afterpulse.ts)), axis=0)
flip_xs = np.concatenate((flip_xs, np.array(sol_afterpulse.ys)), axis=0)

# get growth rates
_, flip_ls, _, _, _, _, _, _ =cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(flip_ts, flip_xs, par_flip, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                             circuit_eff_m_het_div_k_het)

In [10]:
# BOTH TOGGLES FLIPPED - PLOTS

# TOGGLE PROTEIN LEVELS
flip_togprot_figure = bkplot.figure(
        frame_width=200,
        frame_height=125,
        x_axis_label="Time since mutation, h",
        y_axis_label="Protein conc., nM",
        x_range=(-5,tf_afterpulse[1]-tf_pulse[0]),
        y_range=(0,7e4),
        tools="box_zoom,pan,hover,reset,save"
    )

# add shading to show when synthetic gene expression loss occurs
flip_togprot_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_pulse[1] - tf_pulse[0], tf_pulse[1] - tf_pulse[0]],
                            ys=[0, 2e5, 2e5, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
flip_togprot_figure.add_layout(bkmodels.Label(x=0, y=7e4,
                                x_offset=0, y_offset=-12,
                                text='Toggle 12 & 22 inducer pulse',
                                text_font_size='8pt',
                                text_align='left'))

# plot the toggle switch protein concentrations for tog11 and tog21
flip_togprot_figure.line(flip_ts - tf_pulse[0], flip_xs[:, circuit_name2pos['p_tog11']], line_width=2, line_color=bkRGB(100, 149, 237),
           legend_label='tog11')
flip_togprot_figure.line(flip_ts - tf_pulse[0], flip_xs[:, circuit_name2pos['p_tog21']], line_width=2, line_color=bkRGB(207, 181, 59),
           legend_label='tog21')

# plot the toggle switch protein concentrations for tog12 and tog22
flip_togprot_figure.line(flip_ts - tf_pulse[0], flip_xs[:, circuit_name2pos['p_tog12']], line_width=2, line_color=bkRGB(127, 255, 212),
           legend_label='tog12')
flip_togprot_figure.line(flip_ts - tf_pulse[0], flip_xs[:, circuit_name2pos['p_tog22']], line_width=2, line_color=bkRGB(252, 194, 0),
           legend_label='tog22')

# legend formatting
flip_togprot_figure.legend.location = "bottom_right"
flip_togprot_figure.legend.label_text_font_size = "8pt"
flip_togprot_figure.legend.padding = 3
flip_togprot_figure.legend.spacing = 2
flip_togprot_figure.legend.glyph_width = 10
flip_togprot_figure.legend.margin = 15


# set fonts
flip_togprot_figure.xaxis.axis_label_text_font_size = "8pt"
flip_togprot_figure.xaxis.major_label_text_font_size = "8pt"
flip_togprot_figure.yaxis.axis_label_text_font_size = "8pt"
flip_togprot_figure.yaxis.major_label_text_font_size = "8pt"
flip_togprot_figure.yaxis[0].formatter = bkmodels.PrintfTickFormatter(format="%.1e")

# GROWTH RATES AND INTEGRASE LEVELS
flip_grint_figure = bkplot.figure(
        frame_width=200,
        frame_height=125,
        x_axis_label="Time since mutation, h",
        y_axis_label="Cell growth rate, 1/h",
        x_range=(-5, tf_afterpulse[1] - tf_pulse[0]),
        y_range=(1.2, 1.55),
        tools="box_zoom,pan,hover,reset,save"
    )

flip_grint_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_pulse[1] - tf_pulse[0], tf_pulse[1] - tf_pulse[0]],
                            ys=[0, 1.75, 1.75, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
flip_grint_figure.add_layout(bkmodels.Label(x=0, y=1.55,
                                x_offset=0, y_offset=-12,
                                text='Toggle 12 & 22 inducer pulse',
                                text_font_size='8pt',
                                text_align='left'))
# plot the growth rate
flip_grint_figure.line(flip_ts - tf_pulse[0], flip_ls, line_width=2, line_color=bkRGB(0, 0, 0), legend_label='Growth rate')

# create an extra  y range for plotting integrase protein concentrations
flip_grint_figure.extra_y_ranges = {"p_int": bkmodels.Range1d(start=0, end=125)}
flip_grint_figure.add_layout(bkmodels.LinearAxis(y_range_name="p_int",
                                     axis_label="Integrase conc., nM",
                                     axis_line_color=bkRGB(255, 103, 0),
                                     major_tick_line_color=bkRGB(255, 103, 0),
                                     minor_tick_line_color=bkRGB(255, 103, 0)),
                 'right')  # add the alternative axis label to the figure

# plot the integrase protein concentrations
flip_grint_figure.line(flip_ts - tf_pulse[0], flip_xs[:, circuit_name2pos['p_int']], line_width=2, line_color=bkRGB(255, 103, 0),
           y_range_name="p_int", legend_label="Int. conc.")

# set fonts
flip_grint_figure.xaxis.axis_label_text_font_size = "8pt"
flip_grint_figure.xaxis.major_label_text_font_size = "8pt"
flip_grint_figure.yaxis.axis_label_text_font_size = "8pt"
flip_grint_figure.yaxis.major_label_text_font_size = "8pt"

# legend formatting
flip_grint_figure.legend.location = "bottom_right"
flip_grint_figure.legend.label_text_font_size = "8pt"
flip_grint_figure.legend.padding = 3
flip_grint_figure.legend.spacing = 2
flip_grint_figure.legend.glyph_width = 10
flip_grint_figure.legend.margin = 15

# show plots
flip_togprot_figure.output_backend = "svg"
flip_grint_figure.output_backend = "svg"
bkplot.show(bklayouts.column([flip_togprot_figure, flip_grint_figure]))

# save plots
# flip_togprot_figure.output_backend = "svg"
# flip_grint_figure.output_backend = "svg"
# bkplot.output_file("twotoggle_flip_plots.html")
# bkplot.save(bklayouts.column([flip_togprot_figure, flip_grint_figure]))